Notebook to clip Planet mosaic images into image chunks of required size by the crop boundary deep learning model, and remove chunks outside AOIs or with no crop using DE Africa's crop mask 2019.

## Load packages and modules

In [9]:
from glob import glob
import os
import subprocess
import rasterio
import datacube
import numpy as np
import geopandas as gpd
from shapely.geometry import box

## Set parameters

In [10]:
# input shapefile of AOI used for downloading Planet images
AOI_path='input_data/Rwanda_Boundary.shp'

# input folder for the mosaic images
input_folder='results'

# output folder to store image chunks
out_folder=input_folder+'/RGB_chunks'

# country/region name
country='Rwanda'

# year of data
str_year='2021'

# tiling parameters
tile_size=256 # chunk size

overlap=4 # number of overlapping pixels between chunks

In [11]:
if not os.path.isdir(out_folder):
    os.makedirs(out_folder)

## Do clipping into chunks and abandon those totally outside AOI or with no crops

In [12]:
images=glob(input_folder+'/'+country+'*_mosaic_clipped.tif')# search files
print('found {} mosaic images'.format(len(images)))

found 6 mosaic images


In [13]:
AOIs=gpd.read_file(AOI_path)
AOIs

,ADM0_CODE,ADM0_NAME,CONTINENT,ISO3,ISO2,UNI,UNDP,FAOSTAT,GAUL,RIC_ISO3,REC_ISO3,HIH,geometry
0,205,Rwanda,Africa,RWA,RW,646.0,RWA,184.0,205.0,ICPAC-RWA,OTHER-RWA,1,"POLYGON ((30.46679 -1.06294, 30.46446 -1.06678..."


- Extract chunk window (starting from left-top coner of mosaic image)
- Query DE Africa crop mask within window only when chunk window intersects AOI
- Export chunk as RGB bands raster only when crop exists within chunk

In [14]:
%%time
for image in images: # loop through all monthly mosaics
    print('clipping file',image,'into chunks...')
    
    # read in and get information of the mosaic
    ds = rasterio.open(image)
    crs=ds.crs.to_string()
    # x_min,y_min,x_max,y_max=ds.bounds
    width,height=ds.width,ds.height
    AOIs=AOIs.to_crs(crs)
    
    id_column=0 # record column id 
    for column_start in range(0, width-tile_size+overlap, tile_size-overlap):
        id_column+=1
        id_row=0 # record row id
        for row_start in range(0, height-tile_size+overlap, tile_size-overlap):
            id_row+=1
            
            # calculate clipping window
            row_end,column_end=row_start+tile_size,column_start+tile_size

            # load DE Africa crop mask 2019
            x_min,y_max=ds.xy(row_start,column_start)
            x_max,y_min=ds.xy(row_end,column_end)
            
            # get chunk bounding box
            chunk_geom=box(x_min,y_min,x_max,y_max)

            for index, row in AOIs.iterrows():# only do clipping when quad intersects with AOI
                AOI=AOIs.iloc[[index]]
                if chunk_geom.intersects(AOI.geometry.iloc[0]):
#                     print('intersecting')
                    dc = datacube.Datacube(app='cropland_extent')
                    query = {
                        'time': ('2019'),
                        'x': (x_min,x_max),
                        'y': (y_min,y_max),
                        'resolution':(-10, 10),
                        'crs':crs,
                    }
                    cm = dc.load(product='crop_mask',**query).squeeze()
                    np_crop_mask=cm['mask'].to_numpy()

                    # only do clipping and exporting if crop exists in the chunk
                    if np.nansum(np_crop_mask)>0:
                        # NOTE: increase zfill when needed
                        outname=os.path.basename(image)
                        outname=outname[:outname.index('_mosaic_clipped')]
                        outname=os.path.join(out_folder,'_'.join([outname,str(id_row).zfill(3),str(id_column).zfill(3)])+'.tif')
                        if not os.path.exists(outname):
                            # print('writing to file ',outname)
                            gdal_cmd=["gdal_translate", "-of", "GTiff", "-b","1","-b","2","-b","3",'-co','COMPRESS=DEFLATE',
                            "-srcwin",str(column_start),str(row_start),str(tile_size),str(tile_size),image,outname]
                            subprocess.call(gdal_cmd,stdout=subprocess.DEVNULL)
                    break
    ds=None

clipping file results/Rwanda_planet_medres_visual_2021_11_mosaic_clipped.tif into chunks...


2023/02/27 10:37:15.0795: [20314]:  WARNING:       mongoc: Falling back to malloc for counters.


clipping file results/Rwanda_planet_medres_visual_2021_04_mosaic_clipped.tif into chunks...


2023/02/27 11:40:03.0481: [20314]:  WARNING:       mongoc: Falling back to malloc for counters.


clipping file results/Rwanda_planet_medres_visual_2021_08_mosaic_clipped.tif into chunks...


...

clipping file results/Rwanda_planet_medres_visual_2021_12_mosaic_clipped.tif into chunks...
clipping file results/Rwanda_planet_medres_visual_2021_03_mosaic_clipped.tif into chunks...


2023/02/27 18:36:12.0013: [  601]:  WARNING:       mongoc: Falling back to malloc for counters.
2023/02/27 18:53:02.0358: [  601]:  WARNING:       mongoc: Falling back to malloc for counters.
2023/02/27 19:20:03.0550: [20314]:  WARNING:       mongoc: Falling back to malloc for counters.
2023/02/27 19:35:43.0846: [20314]:  WARNING:       mongoc: Falling back to malloc for counters.


clipping file results/Rwanda_planet_medres_visual_2021_10_mosaic_clipped.tif into chunks...


2023/02/27 21:32:22.0840: [  601]:  WARNING:       mongoc: Falling back to malloc for counters.
2023/02/27 22:27:29.0531: [20314]:  WARNING:       mongoc: Falling back to malloc for counters.


CPU times: user 16h 34min 10s, sys: 22h 15min 25s, total: 1d 14h 49min 36s
Wall time: 13h 15min 14s


In [ ]:
# # test if mosaic correctly removes unwanted chunks
# ! gdal_merge.py -o ../experiments/Mozambique_RGB/clipped/mosaic_test.tif -co COMPRESS=Deflate ../experiments/Mozambique_RGB/clipped/*.tif